In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

from compute_ET_green import compute_et_green

from utils import ee_utils

import ee
import geemap

In [3]:
ee.Initialize(project="thurgau-irrigation")

## Importing all assets: land use maps, vegetation periods, WaPOR ET data.

In [4]:
years_to_process = [2018, 2019, 2020, 2021, 2022] # Still need land use for 2023

nutzung_collection = ee.FeatureCollection("projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area")

double_cropping_image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/vegetation_periods_2018-2023")


WaPOR_assets = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal"
WaPOR_et_collections = ee_utils.merge_collections(years=years_to_process, asset_name=WaPOR_assets)

In [5]:
double_cropping_image_2018 = double_cropping_image_collection.first()

## Filtering out potentially irrigated crops and refeence rainfed fields

In [6]:
# Add double cropping information to the feature collection
nutzung_collection_with_double_crop = add_double_cropping_info(
    nutzung_collection, double_cropping_image_2018
)

not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

potentially_rainfed, rainfed_fields = filter_crops(
    nutzung_collection_with_double_crop, exclude_filter, rainfed_filter
)

print("Filtered fields count:", potentially_rainfed.size().getInfo())
print("Rainfed reference fields count:", rainfed_fields.size().getInfo())

Filtered fields count: 12931
Rainfed reference fields count: 21958


## Computing ET green

In [7]:
et_image = WaPOR_et_collections.first()

et_green_result = compute_et_green(et_image, rainfed_fields)

In [8]:
print(et_image.projection().nominalScale().getInfo())
print(et_image.date().format().getInfo())

10
2018-01-01T00:00:00


In [9]:
et_image.bandNames().getInfo()

['downscaled']

In [10]:
print(et_green_result.getInfo())    

{'type': 'Image', 'bands': [{'id': 'ET_green', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 8, 'max': 8}, 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 8.668023839526088, 0, 8.983152841195215e-05, 47.69631950491123]}], 'properties': {'system:time_start': 1514764800000}}


In [11]:
et_green_result.projection().nominalScale().getInfo()

10

In [12]:
print(et_green_result.date().format().getInfo())

2018-01-01T00:00:00
